In [69]:
import os
os.chdir("x:\DL\Projects\MLOPs")
%pwd

'x:\\DL\\Projects\\MLOPs'

In [70]:
print(type(0.01))

<class 'float'>


In [71]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data: Path
    unzip_dir: Path

@dataclass(frozen=True)
class DataValidationConfig:
    current_dset: Path
    root_dir: Path
    status_file_dir: Path
    req_files: list
    
@dataclass(frozen=True)
class TrainLogConfig:
    model: str
    save_path: Path
    mlflow_uri: str
    experiment_name: str
    model_name: str

@dataclass(frozen=True)
class Params:
    optimizer: str
    lr0: float
    save_period: int
    batch: int
    epochs: int
    resume: bool
    seed: int
    imgsz: int 

In [72]:
from scripts.MLOPs.constants import *
from scripts.MLOPs.utils.common import read_yaml, create_directories
#from scripts.MLOPs.entity.config_entity import *

class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_dataingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            local_data= config.local_data,
            unzip_dir= config.unzip_dir
            )
        return data_ingestion_config
    
    def get_datavalidation_config(self)->DataValidationConfig:
        config = self.config.data_validation
        create_directories([config.data_val_dir])
        data_validation_config = DataValidationConfig(
            current_dset= config.current_dset,
            root_dir=config.data_val_dir,
            status_file_dir= config.data_val_status,
            req_files= config.data_val_req
            )
        return data_validation_config
    
    def get_train_log_config(self)-> TrainLogConfig:
        config = self.config.train_log_config
        trainlogconfig = TrainLogConfig(
            model= config.model,
            save_path= config.save_path,
            mlflow_uri= config.mlflow_uri,
            experiment_name= config.experiment_name,
            model_name= config.model_name
        )
        return trainlogconfig
    
    def get_params(self)-> Params:
        param = self.config.param
        params = Params(
            optimizer = param.optimizer,
            lr0 = param.lr0,
            save_period = param.save_period,
            batch = param.batch,
            epochs = param.epochs,
            resume = param.resume,
            seed = param.seed,
            imgsz = param.imgsz
        )
        return params
    
    

In [73]:

# Define a wrapper class to ensure pytorch_model is an instance of torch.nn.Module
import torch
import torch.nn as nn

class WrapperModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, *args, **kwargs):
        return self.model(*args, **kwargs)
    def predict(self, context, model_input):
        # Convert the DataFrame back to a dictionary to pass to the YOLO model
        data_dict = model_input.to_dict(orient="records")[0]
        return self.model(**data_dict)


def convert_pt_to_pytorch_model(model_path):
    # Load the .pt model
    model = torch.load(model_path)
    # Convert it to a PyTorch model if necessary
    # For example:
    # pytorch_model = YOLOl(your_model_arguments)
    # Copy the parameters from the loaded model to the PyTorch model
    # pytorch_model.load_state_dict(model.state_dict())
    return model

from ultralytics import YOLO

def on_train_end(trainer):
        print('Training ended, logging artifacts...')
        best_model_path = trainer.best
        mlflow.log_artifact(best_model_path, "model")
        
        # Register the model in MLflow Model Registry
        mlflow.register_model(
            f"runs:/{mlflow.active_run().info.run_id}/model",
            "my_registered_model"
        )



In [74]:
import logging
import numpy as np
import pandas as pd
import mlflow
from ultralytics import YOLO
from scripts.MLOPs.utils.common import get_highest_train_folder


class YoloWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self):
        self.results = None
        self.results_df = None
        self.data = None

    def load_context(self, context: object):
        """Load Yolo model from context path

        Args:
            context (object): An MLFlow object that is used to define the path to the model.
        """
        runs_f = get_highest_train_folder("runs/detect")
        #logging.info(f"artifacts[path]: runs/detect/{runs_f}/weights/best.pt")
        self.model = YOLO(f"runs/detect/{runs_f}/weights/best.pt")

    def reformat_data(self):
        """Reformat given dictionary object that was coerced into numpy arrays into str,int,flow"""
        # For each key-value pair, convert value to string.
        for key, value in self.data.items():
            if isinstance(value, np.ndarray):
                self.data[key] = ",".join(map(str, [value]))

        # For each key-value pair, convert value to appropriate type.
        for key, value in self.data.items():
            if value.isnumeric():  # Check if the value is an integer
                self.data[key] = int(value)
            elif value.replace(".", "", 1).isdigit():  # Check if the value is a float
                self.data[key] = float(value)
            elif value.lower() in ["true", "false"]:  # Check if the value is a boolean
                self.data[key] = value.lower() == "true"

    def yolo_results_to_df(self):
        """Create Yolo results as a df"""
        # Retrieve bounding boxes
        boxes = self.results[0].boxes
        # Map class to string names
        names = []
        for object_class in boxes.cls.numpy():
            names.append(self.model.names[object_class])
        # Create return df
        self.results_df = pd.DataFrame(
            np.c_[boxes.xyxy.numpy(), boxes.conf, boxes.cls.numpy(), np.array(names)],
            columns=["X1", "Y1", "X2", "Y2", "conf", "cls", "names"],
        )

    def predict(self, context: object, data: dict):
        """Wrapper function around Yolo's predict function. Results are returned as a pandas dataframe.

        Args:
            context (object): An MLFlow object that is used to define the path to the model.
            data (dict): dictionary with source for inference and override parameters for the model.

        Returns:
            _type_: _description_
        """
        self.data = data
        logging.info(f"Data input: f{self.data}")
        # Reformat data
        self.reformat_data()
        logging.info(f"Data after reformat: f{self.data}")

        # Pass inputs to predict
        self.results = self.model.predict(**self.data)
        # Transform results to pandas df
        self.yolo_results_to_df()

        return self.results_df

In [75]:
from ultralytics import YOLO, settings
import os,sys
import logging
import torch
import onnx, onnxruntime
from mlflow.models import infer_signature, validate_serving_input, convert_input_example_to_serving_input
from urllib.parse import urlparse
import mlflow
from scripts.MLOPs.exception import AppException
from scripts.MLOPs.utils.common import get_highest_train_folder, get_random_file_from_folder

class ModelTrainer:
    def __init__(self, config: TrainLogConfig, val: DataValidationConfig, param: Params):
        self.config = config
        self.val = val
        self.param = param

    def validation_status(self):
        with open(self.val.status_file_dir, 'r') as file:
            status = file.read().strip()
        key, value = status.split(':')
        key = key.strip()
        value = value.strip().lower()

        if key != "validation_status":
            raise ValueError("unexpected key in status file")
        
        if value == 'true':
            return True
        elif value == 'false':
            return False
        else:
            raise ValueError("validation status is invalid")
        

    def train_model(self):

        dataset_dir = self.val.current_dset
        data_path = os.path.join( dataset_dir, "data.yaml")
        logging.info(f"Dataset location: {data_path}")
        if torch.cuda.is_available():
            device = torch.cuda.current_device()
            logging.info(f"Device is running on: {torch.cuda.get_device_name(device)}")
        else:
            logging.info(f"CUDA is not available")
            device = "cpu"
            logging.info(f"Device to run on: {device}")
            logging.info(data_path)

        model = self.config.model

        os.makedirs(self.config.save_path, exist_ok=True)
        # save_path = os.path.join(self.config.save_path,"Trainedv8.pt")
        # Load a pretrained YOLOv8n model
        model = YOLO(model)
        # Train the model
        model.train(
            data=data_path,
            optimizer = self.param.optimizer,
            lr0 = self.param.lr0,
            save_period = self.param.save_period,
            batch = self.param.batch,
            epochs = self.param.epochs,
            resume = self.param.resume,
            seed = self.param.seed,
            imgsz = self.param.imgsz
            )
        # savepath = model.save(save_path)
        return model
    
    def log_into_mlflow2(self):
        settings.update({'mlflow': True})
        settings.reset()
        run_name = self.config.model_name
        experiment_name = self.config.experiment_name
        os.environ["MLFLOW_TRACKING_URI"] = self.config.mlflow_uri
        os.environ["MLFLOW_RUN"] = run_name
        print("MLFLOW_TRACKING_URI: ", os.environ.get("MLFLOW_TRACKING_URI"))
        mlflow.set_experiment(experiment_name=experiment_name)
        with mlflow.start_run(run_name=run_name) as run:
            run_id = run.info.run_id
            print(f"Current MLflow Run ID: {run_id}")

            dataset_dir = self.val.current_dset
            data_path = os.path.join( dataset_dir, "data.yaml")
            logging.info(f"Dataset location: {data_path}")
            if torch.cuda.is_available():
                device = torch.cuda.current_device()
                logging.info(f"Device is running on: {torch.cuda.get_device_name(device)}")
            else:
                logging.info(f"CUDA is not available")
                device = "cpu"
                logging.info(f"Device to run on: {device}")
                logging.info(data_path)

            model = self.config.model

            os.makedirs(self.config.save_path, exist_ok=True)
            # save_path = os.path.join(self.config.save_path,"Trainedv8.pt")
            # Load a pretrained YOLOv8n model
            model = YOLO(model)
            model.add_callback("on_train_end", on_train_end)
            # Train the model
            results = model.train(
                data=data_path,
                optimizer = self.param.optimizer,
                lr0 = self.param.lr0,
                save_period = self.param.save_period,
                batch = self.param.batch,
                epochs = self.param.epochs,
                resume = self.param.resume,
                seed = self.param.seed,
                imgsz = self.param.imgsz
                )
            
            mlflow.log_metrics({
                "Precision": results.results_dict['metrics/precision(B)'],
                "Recall": results.results_dict['metrics/recall(B)'],
                "mAP": results.results_dict['metrics/mAP50-95(B)'],
                "mAP50": results.results_dict['metrics/mAP50(B)'],
            })
            
            h_folder = get_highest_train_folder("runs/detect")
            model_artifact = f"runs/detect/{h_folder}"
            pytorch_model = convert_pt_to_pytorch_model(f"{model_artifact}/weights/best.pt")

            # Wrap pytorch_model
            wrapped_model = WrapperModel(pytorch_model)


            # Save the wrapped_model with MLflow
            mlflow.pytorch.log_model(wrapped_model, artifact_path="pytorch_model" ,registered_model_name=run_name)
            mlflow.pytorch.save_model(wrapped_model, path=f"{model_artifact}/pytorch_model",pip_requirements="requirements.txt")
            mlflow.log_artifacts(model_artifact, artifact_path="pytorch_model")

            mlflow.log_param("device", device)
            mlflow.log_param("optimizer", self.param.optimizer)
            mlflow.log_param("learning_rate", self.param.lr0)
            mlflow.log_param("batch_size", self.param.batch)
            mlflow.log_param("epochs", self.param.epochs)

            mlflow.end_run()
    
    def log_into_mlflow(self):

        settings.update({'mlflow': True})
        settings.reset()
        run_name = self.config.model_name
        experiment_name = self.config.experiment_name
        os.environ["MLFLOW_TRACKING_URI"] = self.config.mlflow_uri
        os.environ["MLFLOW_RUN"] = run_name
        
        print("MLFLOW_TRACKING_URI: ", os.environ.get("MLFLOW_TRACKING_URI"))
        
        # settings.update({'datasets_dir': "X:\DL\Projects\MLOPs"})
        mlflow.pytorch.autolog(log_models=True)
        mlflow.set_experiment(experiment_name=experiment_name)
        
        with mlflow.start_run(run_name=run_name) as run: 
            #run_name is the name of the task.
            run_id = run.info.run_id 
            #run_id is the directory name that will be stored within the MLFLOW_TRACKING_URI path.
            print(run_id)
            save_path = os.path.join(self.config.save_path,"Trainedv8.pt")
            model = YOLO(save_path)
            yolonnx = model.export(format="onnx", dynamic= True)
            yolonnx_model = onnx.load(yolonnx)
            mlflow.onnx.log_model(yolonnx_model, "YOLOv8n")             ##logged model
        mlflow.end_run()
            
    
    
    # def run_pipeline(self):
    #     try:
    #         with open()
    #         self.train_model()
    #     except Exception as e:
    #         raise AppException(e, sys)

    def run_pipeline(self):
        if (self.validation_status() == True):
            try:
                self.log_into_mlflow()
            except Exception as e:
                raise AppException(e, sys)
        else:
            logging.INFO(f"Model training not run due to invalid dataset. \n please ingest a valid dataset")

            


In [76]:
"Registers model in MLFlow"
import os
from pathlib import Path
import csv
import logging
import yaml
import cloudpickle
import pandas as pd
import mlflow
# from scripts.mlflow_utils import model_wrapper
# from scripts.mlflow_utils.model_wrapper import YoloWrapper


def get_experiment_id(name: str):
    """Retrieve experiment if registered name, else create experiment.

    Args:
        name (str): Mlflow experiment name

    Returns:
        str: Mlfow experiment id
    """
    exp = mlflow.get_experiment_by_name(name)
    if exp is None:
        exp_id = mlflow.create_experiment(name)
        return exp_id
    return exp.experiment_id


def read_lines(path: str):
    """Given a path to a file, this function reads file, seperates the lines and returns a list of those separated lines.

    Args:
        path (str): Path to file

    Returns:
        list: List made of of file lines
    """
    with open(path) as f:
        return f.read().splitlines()


def log_metrics(save_dir: str, log_results: bool = True):
    """Log metrics to Mlflow from the Yolo model outputs.

    Args:
        save_dir (str): Path to Yolo save directory, i.e - runs/train
        log_results (bool): If True, the results are logged to MLflow server
    """
    save_dir = Path(save_dir)
    try:
        with open(save_dir / "results.csv", "r") as csv_file:
            metrics_reader = csv.DictReader(csv_file)
            metrics_list = []
            for metrics in metrics_reader:
                # Create an empty dictionary to store the updated key-value pairs for this row
                updated_metrics = {}
                # Iterate through the key-value pairs in this row's dictionary
                for key, value in metrics.items():
                    # Remove whitespace from the key
                    key = key.strip()
                    value = value.strip()
                    # Remove extra strings in keys
                    patterns = ["(B)", "metrics/"]
                    for pattern in patterns:
                        key = key.replace(pattern, "")
                    # Add the updated key-value pair to the updated row dictionary
                    try:
                        # Add the updated key-value pair to the updated row dictionary
                        updated_metrics[key] = float(value)
                    except ValueError:
                        logging.error(f"ValueError: Could not convert {value} to float.")
                    metrics_list.append(updated_metrics)
                    if log_results:
                        mlflow.log_metrics(updated_metrics)
        return metrics_list
    except FileNotFoundError:
        print(f"FileNotFoundError: Could not find {save_dir / 'results.csv'}.")
    except IOError:
        print(f"IOError: Could not read {save_dir / 'results.csv'}.")




def register_model(experiment_name: str, model_name: str, save_dir: Path):
    """Registers a model with mlflow

    Args:
        experiment_name (str): Name of Mlfow experiment
        model_name (str): Name that will be registered with Mlflow
        save_dir (Path): Path object where the results of the Yolo model are saved. I.e 'runs' directory
    """
    save_dir = Path(save_dir)
    logging.debug(f"Save Directory: {save_dir}")

    '''model_path = get_path_w_extension(
        path=save_dir, extension=".pt", limit=1, ignore_files=["last.pt"]
    )[0]'''
    model_path = f"{save_dir}/weights/best.pt"
    artifacts = {"path": model_path}

    model = YoloWrapper()

    exp_id = get_experiment_id(experiment_name)

    #cloudpickle.register_pickle_by_value(model_wrapper)

    with mlflow.start_run(experiment_id=exp_id) as run:
        # Log some params
        with open(save_dir / "args.yaml", "r") as param_file:
            params = yaml.safe_load(param_file)
        mlflow.log_params(params)

        log_metrics(save_dir, True)
        mlflow.log_artifact(f"{save_dir}/weights/best.pt")
        pip_reqs = read_lines("requirements.txt")
        mlflow.pyfunc.log_model(
            "model",
            python_model=model,
            pip_requirements=pip_reqs,
            artifacts=artifacts,
            registered_model_name=model_name,
        )
        run_id = run.info.run_uuid
        experiment_id = run.info.experiment_id
        mlflow.end_run()
        logging.info(f"artifact_uri = {mlflow.get_artifact_uri()}")
        logging.info(f"runID: {run_id}")
        logging.info(f"experiment_id: {experiment_id}")




In [77]:
try:
    xy = ConfigurationManager()
    trainlog = xy.get_train_log_config()
    valc = xy.get_datavalidation_config()
    paramss = xy.get_params()
    x = ModelTrainer(config=trainlog, val=valc,param=paramss)
    x.train_model()

    runs_f = get_highest_train_folder("runs/detect")
    dirr = Path(f"runs/detect/{runs_f}")

    register_model(experiment_name=trainlog.experiment_name,
                   model_name=trainlog.model_name,
                   save_dir=dirr)
except Exception as e:
    raise e


[2024-08-05 11:59:38,438: INFO: common: yaml file: config\config.yaml loaded sucessfully]
[2024-08-05 11:59:38,445: INFO: common: yaml file: params.yaml loaded sucessfully]
[2024-08-05 11:59:38,447: INFO: common: created directory at artifacts]
[2024-08-05 11:59:38,449: INFO: common: created directory at artifacts/data_validation]
[2024-08-05 11:59:38,449: INFO: 4122740354: Dataset location: artifacts/data_ingestion/data\data.yaml]
[2024-08-05 11:59:38,449: INFO: 4122740354: CUDA is not available]
[2024-08-05 11:59:38,451: INFO: 4122740354: Device to run on: cpu]
[2024-08-05 11:59:38,452: INFO: 4122740354: artifacts/data_ingestion/data\data.yaml]
New https://pypi.org/project/ultralytics/8.2.73 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.67  Python-3.9.19 torch-2.4.0+cpu CPU (AMD Ryzen 7 5825U with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=artifacts/data_ingestion/data\data.yaml, epochs=1, time=None, patience=100, batch

train: Scanning X:\DL\Projects\MLOPs\artifacts\data_ingestion\data\train\labels.cache... 624 images, 0 backgrounds, 0 corrupt: 100%|██████████| 624/624 [00:00<?, ?it/s]
val: Scanning X:\DL\Projects\MLOPs\artifacts\data_ingestion\data\valid\labels.cache... 331 images, 0 backgrounds, 0 corrupt: 100%|██████████| 331/331 [00:00<?, ?it/s]

Plotting labels to runs\detect\train63\labels.jpg... 


optimizer: SGD(lr=0.03, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005625000000000001), 63 bias(decay=0.0)
MLflow: logging run_id(49c29f6d4ebf48e9a5c50f3f8da328a6) to http://127.0.0.1:5000
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train63
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      1.053      3.246       1.43         60        640: 100%|██████████| 26/26 [04:00<00:00,  9.25s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:53<00:00,  7.68s/it]


...

2024/08/05 12:05:36 INFO mlflow.tracking._tracking_service.client: 🏃 View run yolov8ndet at: http://127.0.0.1:5000/#/experiments/843375537309377713/runs/49c29f6d4ebf48e9a5c50f3f8da328a6.
2024/08/05 12:05:36 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/843375537309377713.


MLflow: results logged to http://127.0.0.1:5000
MLflow: disable with 'yolo settings mlflow=False'


2024/08/05 12:05:38 INFO mlflow.types.utils: Unsupported type hint: <class 'dict'>, skipping schema inference
Registered model 'yolov8ndet' already exists. Creating a new version of this model...
2024/08/05 12:05:38 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: yolov8ndet, version 11
Created version '11' of model 'yolov8ndet'.
2024/08/05 12:05:38 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/05 12:05:38 INFO mlflow.tracking._tracking_service.client: 🏃 View run fun-gnu-699 at: http://127.0.0.1:5000/#/experiments/275689030631216605/runs/770fcd33baf64af2ba02ffb4d5e90613.
2024/08/05 12:05:38 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/275689030631216605.


[2024-08-05 12:05:38,649: INFO: 2215763116: artifact_uri = mlflow-artifacts:/843375537309377713/e0e995d16c7b4d1a8ebc0fac34bb4b38/artifacts]
[2024-08-05 12:05:38,649: INFO: 2215763116: runID: 770fcd33baf64af2ba02ffb4d5e90613]
[2024-08-05 12:05:38,649: INFO: 2215763116: experiment_id: 275689030631216605]


2024/08/05 12:05:38 INFO mlflow.tracking._tracking_service.client: 🏃 View run efficient-stoat-96 at: http://127.0.0.1:5000/#/experiments/843375537309377713/runs/e0e995d16c7b4d1a8ebc0fac34bb4b38.
2024/08/05 12:05:38 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/843375537309377713.


In [68]:
try:
    obj2 = ConfigurationManager()
    model_config = obj2.get_train_log_config()
    valc = obj2.get_datavalidation_config()
    params = obj2.get_params()
    x = ModelTrainer(config=model_config, val=valc, param=params)
    x.train_model()
    #x.log_into_mlflow2()
    #x.log_into_mlflow()
except Exception as e:
    raise AppException(e, sys)


[2024-08-05 10:12:35,955: INFO: common: yaml file: config\config.yaml loaded sucessfully]
[2024-08-05 10:12:35,961: INFO: common: yaml file: params.yaml loaded sucessfully]
[2024-08-05 10:12:35,963: INFO: common: created directory at artifacts]
[2024-08-05 10:12:35,966: INFO: common: created directory at artifacts/data_validation]
[2024-08-05 10:12:35,968: INFO: 4122740354: Dataset location: artifacts/data_ingestion/data\data.yaml]
[2024-08-05 10:12:35,971: INFO: 4122740354: CUDA is not available]
[2024-08-05 10:12:35,972: INFO: 4122740354: Device to run on: cpu]
[2024-08-05 10:12:35,972: INFO: 4122740354: artifacts/data_ingestion/data\data.yaml]


New https://pypi.org/project/ultralytics/8.2.73 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.67  Python-3.9.19 torch-2.4.0+cpu CPU (AMD Ryzen 7 5825U with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=artifacts/data_ingestion/data\data.yaml, epochs=1, time=None, patience=100, batch=24, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train62, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=

train: Scanning X:\DL\Projects\MLOPs\artifacts\data_ingestion\data\train\labels.cache... 624 images, 0 backgrounds, 0 corrupt: 100%|██████████| 624/624 [00:00<?, ?it/s]
val: Scanning X:\DL\Projects\MLOPs\artifacts\data_ingestion\data\valid\labels.cache... 331 images, 0 backgrounds, 0 corrupt: 100%|██████████| 331/331 [00:00<?, ?it/s]

Plotting labels to runs\detect\train62\labels.jpg... 


optimizer: SGD(lr=0.03, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005625000000000001), 63 bias(decay=0.0)
MLflow: logging run_id(981dec4126304493b1f323a01a1bd72a) to http://127.0.0.1:5000
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train62
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        331        497    0.00238      0.354     0.0502     0.0312

1 epochs completed in 0.091 hours.
Optimizer stripped from runs\detect\train62\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train62\weights\best.pt, 6.2MB

Validating runs\detect\train62\weights\best.pt...
Ultralytics YOLOv8.2.67  Python-3.9.19 torch-2.4.0+cpu CPU (AMD Ryzen 7 5825U with Radeon Graphics)
Model summary (fused): 168 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:47<00:00,  6.83s/it]


                   all        331        497    0.00241      0.354     0.0505     0.0315
              Backpack         95        117          0          0          0          0
             Cellphone         55         76    0.00984      0.592      0.039     0.0198
                 Drill         79         81   0.000886      0.556      0.046     0.0238
     Fire extinguisher         85         95    0.00134      0.621      0.167      0.114
              Survivor        117        128          0          0          0          0
Speed: 3.5ms preprocess, 120.6ms inference, 0.0ms loss, 10.0ms postprocess per image
Results saved to runs\detect\train62


2024/08/05 10:19:06 INFO mlflow.tracking._tracking_service.client: 🏃 View run yolov8ndet at: http://127.0.0.1:5000/#/experiments/843375537309377713/runs/981dec4126304493b1f323a01a1bd72a.
2024/08/05 10:19:06 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/843375537309377713.


MLflow: results logged to http://127.0.0.1:5000
MLflow: disable with 'yolo settings mlflow=False'


In [13]:
try:
    obj2 = ConfigurationManager()
    model_config = obj2.get_train_log_config()
    valc = obj2.get_datavalidation_config()
    params = obj2.get_params()
    x = ModelTrainer(config=model_config, val=valc, param=params)
    x.run_pipeline()
except Exception as e:
    raise AppException(e, sys)

[2024-08-02 17:38:58,388: INFO: common: yaml file: config\config.yaml loaded sucessfully]
[2024-08-02 17:38:58,393: INFO: common: yaml file: params.yaml loaded sucessfully]
[2024-08-02 17:38:58,395: INFO: common: created directory at artifacts]
[2024-08-02 17:38:58,396: INFO: common: created directory at artifacts/data_validation]


2024/08/02 17:38:58 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.


MLFLOW_TRACKING_URI:  http://127.0.0.1:5000
8efebb9c90134454861cd2b4e645ac17
[2024-08-02 17:38:58,905: INFO: 2200128603: Dataset location: artifacts/data_ingestion/data\data.yaml]
[2024-08-02 17:38:58,905: INFO: 2200128603: CUDA is not available]
[2024-08-02 17:38:58,905: INFO: 2200128603: Device to run on: cpu]
[2024-08-02 17:38:58,909: INFO: 2200128603: artifacts/data_ingestion/data\data.yaml]
New https://pypi.org/project/ultralytics/8.2.71 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.67  Python-3.9.19 torch-2.4.0+cpu CPU (AMD Ryzen 7 5825U with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=artifacts/data_ingestion/data\data.yaml, epochs=1, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train44, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=

2024/08/02 17:39:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run yolov8det at: http://127.0.0.1:5000/#/experiments/895467674014368248/runs/8efebb9c90134454861cd2b4e645ac17.
2024/08/02 17:39:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/895467674014368248.


AppException: Error occured in python script name [C:\Users\aravi\AppData\Local\Temp\ipykernel_28176\2631944331.py] line number [7] error message [Error occured in python script name [C:\Users\aravi\AppData\Local\Temp\ipykernel_28176\2200128603.py] line number [108] error message [Dataset 'artifacts/data_ingestion/data/data.yaml' error  'artifacts/data_ingestion/data\data.yaml' does not exist]]

AttributeError: 'ModelTrainer' object has no attribute 'log_into_mlflow'

In [10]:
try:
    obj2 = ConfigurationManager()
    model_config = obj2.get_model_trainer()
    dirc = obj2.get_dataingestion_config()
    x = ModelTrainer(config=model_config, dir=dirc)
    x.train_model()
except Exception as e:
    raise AppException(e, sys)


[2024-07-29 00:59:56,601: INFO: common: yaml file: config\config.yaml loaded sucessfully]
[2024-07-29 00:59:56,604: INFO: common: yaml file: params.yaml loaded sucessfully]
[2024-07-29 00:59:56,604: INFO: common: created directory at artifacts]
[2024-07-29 00:59:56,605: INFO: common: created directory at artifacts/data_ingestion]
[2024-07-29 00:59:56,606: INFO: 1550852978: Dataset location: artifacts/data_ingestion/data\data.yaml]
[2024-07-29 00:59:56,607: INFO: 1550852978: CUDA is not available]
[2024-07-29 00:59:56,607: INFO: 1550852978: Device to run on: cpu]
[2024-07-29 00:59:56,607: INFO: 1550852978: artifacts/data_ingestion/data\data.yaml]
New https://pypi.org/project/ultralytics/8.2.68 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.67  Python-3.9.19 torch-2.4.0+cpu CPU (AMD Ryzen 7 5825U with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=artifacts/data_ingestion/data\data.yaml, epochs=1, time=None, patience=100, batch=

c:\Users\aravi\anaconda3\envs\MLOPs\lib\site-packages\ultralytics\engine\trainer.py:268: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)
train: Scanning X:\DL\Projects\MLOPs\artifacts\data_ingestion\data\train\labels... 2326 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2326/2326 [00:06<00:00, 354.57it/s]


train: New cache created: X:\DL\Projects\MLOPs\artifacts\data_ingestion\data\train\labels.cache


val: Scanning X:\DL\Projects\MLOPs\artifacts\data_ingestion\data\valid\labels... 331 images, 0 backgrounds, 0 corrupt: 100%|██████████| 331/331 [00:01<00:00, 272.30it/s]


val: New cache created: X:\DL\Projects\MLOPs\artifacts\data_ingestion\data\valid\labels.cache
Plotting labels to runs\detect\train10\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005078125), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train10
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      1.326      2.858      1.536          5        640: 100%|██████████| 466/466 [12:00<00:00,  1.55s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:42<00:00,  1.25s/it]


                   all        331        497      0.499      0.537      0.502      0.245

1 epochs completed in 0.213 hours.
Optimizer stripped from runs\detect\train10\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train10\weights\best.pt, 6.2MB

Validating runs\detect\train10\weights\best.pt...
Ultralytics YOLOv8.2.67  Python-3.9.19 torch-2.4.0+cpu CPU (AMD Ryzen 7 5825U with Radeon Graphics)
Model summary (fused): 168 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:34<00:00,  1.03s/it]


                   all        331        497      0.513      0.529      0.505      0.246
              Backpack         95        117      0.721      0.575      0.706      0.359
             Cellphone         55         76      0.395      0.474       0.36      0.214
                 Drill         79         81      0.394      0.469      0.408      0.162
     Fire extinguisher         85         95      0.468      0.574      0.507      0.211
              Survivor        117        128      0.589      0.555      0.542      0.286
Speed: 2.5ms preprocess, 84.9ms inference, 0.0ms loss, 8.2ms postprocess per image
Results saved to runs\detect\train10
